# RNN (Pytorch 첫걸음)

일반적인 신경망과 다르게, 내부 상태를 저장하고 있다. 특정 시점 t의 입력값 x(t)와 이전 시점의 내부상태 h(t-1)을 입력하면 새로운 내부상태 h(t)를 출력.

일반 신경망보다 train이 어렵다. 오랜 시간 쌓인 이력을 사용한다는 건 그만큼 layer가 deep하다는 것. gradient vanishing이나 경사 분실 문제 발생 가능.


따라서 RNN의 layer를 단순 선형(누적형태) 대신 정교한 처리 모듈로 변경한 LSTM이나 GRU 등의 RNN 모듈도 있다.

## 텍스트 데이터의 수치화

세 단계로 구성
    1. 정규화 & 토큰화
    2. Dictionary 구축
    3. 수치로 변환

1. 문장을 특정 단위의 리스트로 분해한다. 예컨대 단어나 문자 등을 분할 단위로 사용. 유럽계 언어는 공백으로 구분해도 충분하지만 일본어, 중국어 등은 형태소 분석 처리가 필요하기도 함. 표기 차이도 통일해야 함. 예컨대 소문자로 전부 통일 / ~한다, ~하다 = ~하다로 통일. isn't 는 is not으로 통일하는 등.

2. 모든 문장의 집합(Corpus)에 대한 토큰을 수집하고, 숫자 id를 부여하는 작업. 등장 순서대로 부여하거나 빈도수에 따라 부여하거나.

3. 토큰의 리스트로 분할된 문장을 Dictionary를 사용해 id list로 변환한다.

이 작업을 거치며 하나의 긴 문자열이었던 문장이 '수치 리스트'로 변한다. 이 리스트를 다시 집계하고 id의 등장횟수를 벡터로 표현한 것이 BoW.

ex) (I, you, am, of ...) = (1,0,1,3 ..)

BoW는 계산이 간단하고, 여러 문장을 모으면 희소행렬로 표현할 수 있어 효율은 좋다. 단 토큰 순서 정보를 잃는다.

신경망에서는 Embedding이라는 기법으로 토큰을 벡터화하고, 벡터 데이터의 시계열로 문장을 처리하는 것이 주류임.

Pytorch는 nn.Embedding으로 layer 생성이 가능하다.


```python
# 전체 10,000 종류의 토큰을 20차원 벡터로 표현하는 경우
emb = nn.Embedding(10000, 20, padding_idx = 0)
# Embedding layer input타입은 int64
inp = torch.tensor([1,2,5,2,10], dtype = torch.int64)
# 출력은 float32
out = emb(inp)
```

padding_idx를 지정하므로, 이 경우 id가 0인 벡터로 바뀐다. 사전에 없는 토큰은 id가 0, 실제 id는 1부터 시작하도록 세팅한다.

토큰 종류는 0을 포함한 수를 nn.Embedding의 첫 번째 인수로 지정해야 한다.


(nn.Embedding 값도 미분 가능. 내부의 가중치 파라미터의 학습이 가능하다는 의미다. Neural Net 학습 시 여기도 최적화가 가능함. 사전에 학습된 nn.Embedding 값에 기반한 transfer Learning 가능)  -- ex) Word2Vec으로 유명한 Continuous-BOW나 Skip-Gram 등의 모델을 쓰는 경우

# IMDB 영화평점 평가 데이터 - 긍정 / 부정 이진분류

In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz

--2019-05-20 07:22:23--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  36.7MB/s    in 2.2s    

2019-05-20 07:22:25 (36.7 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

In [0]:
def text_to_ids(text, vocab_dict):
  # ?! 이외의 기호 삭제
  text = remove_marks_regex.sub("",text)
  # ?!와 단어 사이 공백
  text = shift_marks_regex.sub(r" \1 ",text)
  
  tokens = text.split()
  return [vocab_dict.get(token,0) for token in tokens]

# 긴 문자열을 토큰 id 리스트로 변환하는 함수. 정규식으로 문장부호 및 괄호 제거, ?, ! 사이에 공백 넣어 단어와 별도로 토큰 분할
# imdb.vocab에 ?와 !가 포함되어 있기 때문
# 용어집에 포함되지 않은 토큰은 0으로 할당한다.


In [0]:
def list_to_tensor(token_idxes, max_len=100, padding=True):
  if len(token_idxes) > max_len:
    token_idxes = token_idxes[:max_len]
  n_tokens = len(token_idxes)
  if padding:
    token_idxes = token_idxes + [0] * (max_len - len(token_idxes))
  return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

# id 리스트를 int64 tensor로 변환하는 함수. 각 문장을 분할한 후 토큰 수를 제한하고,
# 제한 숫자에 미치지 못할 경우 0으로 뒤를 채운다

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

In [0]:
class IMDBDataset(Dataset):
  @property
  def vocab_size(self):
    return len(self.vocab_array)
  def __len__(self):
    return len(self.labeled_files)

  def __getitem__(self, idx):
    label, f = self.labeled_files[idx]
    # 파일의 텍스트 데이터를 읽고 소문자로 변환
    data = open(f).read().lower()

    # 텍스트 데이터를 id 리스트로 변환
    data = text_to_ids(data, self.vocab_dict)
    data, n_tokens = list_to_tensor(data, self.max_len, self.padding)
    return data, label, n_tokens
  
  def __init__(self, dir_path, train=True, max_len = 100, padding=True):
    self.max_len = max_len
    self.padding = padding
    
    path = pathlib.Path(dir_path)
    vocab_path = path.joinpath("imdb.vocab")
    
    # 용어집 파일 읽고 행 단위로 분할
    self.vocab_array = vocab_path.open().read().strip().splitlines()
    
    # 단어가 key고 값이 ID인 dict 만들기
    self.vocab_dict = dict((w, i+1) for (i,w) in enumerate(self.vocab_array))
    
    if train:
      target_path = path.joinpath("train")
    else:
      target_path = path.joinpath("test")
      
    pos_files = sorted(glob.glob(str(target_path.joinpath("pos/*.txt"))))
    neg_files = sorted(glob.glob(str(target_path.joinpath("neg/*.txt"))))
    
    # pos는 1, neg는 0 label 붙여서 (file_path, label)의 튜플리스트 작성
    self.labeled_files = list(zip([0]*len(neg_files), neg_files)) + list(zip([1]*len(pos_files), pos_files))
    


In [0]:
train_data = IMDBDataset("./aclImdb/")
test_data = IMDBDataset("./aclImdb/", train=False)

In [0]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size = 32, shuffle=False, num_workers=4)

해결하려는 문제: "특정 정수의 시계열 X가 입력되었을 때 0 or 1이 출력되는 이진 분류 문제." 

즉 시계열로 문자열을 정수로 변환한 데이터가 들어올 때, 최종적으로 '긍정 / 부정' 분류하느 ㄴ것.

== 입력 X를 Embedding으로 벡터 시계열로 변환한 후, RNN에 넣어서 마지막 출력을 1차원 선형 계층으로 연결하면 된다.

In [0]:
class SequenceTaggingNet(nn.Module):
  
  
  def __init__(self, num_embeddings, 
               embedding_dim = 50, 
               hidden_size=50, 
               num_layers=1,
              dropout=.2):
    super().__init__()
    
    self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx = 0)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x, h0 = None, l = None):
    # ID -> 임베딩으로 다차원 벡터 변환
    # x는 (batch size, step_size)
    # --> (batch_size, step_size, embedding_dim)
    x = self.emb(x)
    # 초기 상태 h0와 함께 RNN에 X 전달
    # (batch_size, step_size, embedding_dim) -> ( , , hidden_dim)
    x, h = self.lstm(x, h0)
    
    # 마지막 단계 = (batch_size, step_size, hidden_dim) -> (batch_size, 1)
    if l is not None:
      # 입력 원래 길이가 있으면 이용
      x = x[list(range(len(x))),l-1,:]
    else:
      # 없으면 마지막 것
      x = x[:,-1,:]
    
    # 추출한 마지막 단계를 선형 레이어에 넣는다
    x = self.linear(x)
    # 불필요한 차원 제거
    # (batch_size, 1) = (batch_size ,)
    x = x.squeeze()
    return x


입력을 매 단계 RNN에 직접 넣을 필요 없음. 

pytorch의 RNN 계열은 여러 단계의 input 받은 뒤 여러 단계 출력 & 마지막 내부 상태를 반환하도록 설계됨

---

입력 dim, 중간층 dim 차원 외에도 num_layers (layer 수), batch_first, dropout 등의 인수 저장.

batch_first를 쓰면, (step_size, batch_size, dim) 기본 설정 대신 (batch_size, step_size, dim) 형태로 변경 가능.

---

forward 함수에서 내부 상태 초기값 지정이 필요한데, None으로 설정할 경우 모든 값이 0인 벡터를 입력한 것과 같은 효과.

마지막 단계만 선형 layer에 전달... (batch_size, 1) -> (batch_size, )로 변경. 이진 분류에서 사용하는 형태로 변환한다.



## Validate

In [0]:
def eval_net(net, data_loader, device = 'cpu'):
  net.eval()
  ys = []
  ypreds = []
  for x, y, l in data_loader:
    x = x.to(device)
    y = y.to(device)
    l = l.to(device)
    with torch.no_grad():
      y_pred = net(x, l = l)
      y_pred = (y_pred >0).long()
      ys.append(y)
      ypreds.append(y_pred)
  
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  acc = (ys == ypreds).float().sum() / len(ys)
  return acc.item()


## Training

In [0]:
from numpy import mean
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
# num_embedding에는 0을 포함해서 train_data.vocab.size+1 을 넣는다.
net = SequenceTaggingNet(train_data.vocab_size+1, num_layers = 2)
net.to(device)
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
  losses = []
  net.train()
  
  for x, y, l in tqdm.tqdm(train_loader):
    x = x.to(device)
    y = y.to(device)
    l = l.to(device)
    y_pred = net(x, l = l)
    loss = loss_f(y_pred, y.float())
    
    net.zero_grad()
    loss.backward()
    opt.step()
    
    losses.append(loss.item())
    
  train_acc = eval_net(net, train_loader, device)
  val_acc = eval_net(net, test_loader, device)
  print(epoch+1, mean(losses), train_acc, val_acc)
  


  0%|          | 0/782 [00:00<?, ?it/s]

1 0.6870950050366199 0.5941599607467651 0.5903199911117554



  0%|          | 0/782 [00:00<?, ?it/s]

2 0.6454496995719803 0.6995599865913391 0.6796799898147583



  0%|          | 0/782 [00:00<?, ?it/s]

...

8 0.2097501949647732 0.9501999616622925 0.790399968624115



  0%|          | 0/782 [00:00<?, ?it/s]

9 0.16237005217672537 0.955079972743988 0.7831599712371826



100%|██████████| 782/782 [00:16<00:00, 51.71it/s]


10 0.1302645599929249 0.9737199544906616 0.7856400012969971
